In [3]:
import numpy as np
import pandas as pd
import folium
import h3.api.numpy_int as h3
from folium.vector_layers import CircleMarker

In [2]:
def get_hexagon(h):
    geo_lst = list(h3.h3_to_geo_boundary(h))
    geo_lst.append(geo_lst[0])
    return np.array(geo_lst)

In [3]:
center = [38.72533, -9.15]

In [4]:
m = folium.Map(location=center, zoom_start=18)

hh = h3.geo_to_h3(38.72533, -9.15, 11)
ring = h3.k_ring(hh, 2)

for r in ring:
    hexagon = get_hexagon(r)
    polygon = folium.vector_layers.Polygon(locations=hexagon)
    polygon.add_to(m)
m

## Sorted Search Demonstration

In [5]:
data = np.array([7, 7, 2, 7, 7, 3, 3, 2, 3, 8, 8, 8, 2, 8, 8, 5, 6, 5, 6, 5, 6])
idxs = np.argsort(data)

keys = np.array([1, 2, 3])

In [7]:
i0 = np.searchsorted(data, keys, side="left", sorter=idxs)
i0

array([0, 0, 3])

In [8]:
i1 = np.searchsorted(data, keys, side="right", sorter=idxs)
i1

array([0, 3, 6])

In [10]:
data[idxs[0:3]], idxs[0:3]

(array([2, 2, 2]), array([ 2,  7, 12]))

In [11]:
data[idxs[3:6]], idxs[3:6]

(array([3, 3, 3]), array([5, 6, 8]))

## Performance

In [15]:
data_dict = {
    "init_geo": [29.76275701, 28.861421639000003, 
                 29.021140319, 28.71507979, 
                 28.807593859, 29.268012094, 
                 28.818752568999997, 28.573495078, 
                 28.625998347000003, 29.172096508000003],
    "init_h3i": [29.002119281999995, 31.501784377000007, 
                 29.913000368000006, 30.084877427000002, 
                 29.115711202, 29.365764047, 
                 29.375851799000003, 30.04566253, 
                 29.513744804999995, 30.07629952],
    "qrad_geo": [14.455405539000004, 12.925163823999995, 
                 14.692950778999993, 16.122003127000006, 
                 18.081273656, 16.782265988000006, 
                 13.358665484999996, 14.337435540000001, 
                 16.600762007, 15.374777615],
    "qrad_h3i": [5.463111709999993, 3.119011846999996, 
                 5.1980214820000015, 6.469879312000003, 
                 6.439164724000008, 6.789193827000005, 
                 4.794309327999997, 4.683093052000004, 
                 6.548858373999991, 6.293878848000006],
    "qknn_geo": [7.1081926910000135, 6.342480156999997, 
                 7.460930053999988, 7.468806442999991, 
                 8.582897841000005, 8.199856435000001, 
                 6.301281680999992, 7.061586293000005, 
                 7.844416269000007, 7.850034356999998],
    "qknn_h3i": [5.567441251999995, 3.4514994520000073, 
                 5.014784836000004, 6.786518560999994, 
                 6.629357628999998, 7.221533246999996, 
                 4.706059526000004, 4.923216506000003, 
                 6.839402250000006, 6.238044574999989]
}

In [16]:
perf_df = pd.DataFrame.from_dict(data_dict)

In [21]:
geo_idx = perf_df.mean().index.str.endswith("geo")
h3i_idx = perf_df.mean().index.str.endswith("h3i")

In [22]:
perf_df.mean()[geo_idx]

init_geo    28.962635
qrad_geo    15.273070
qknn_geo     7.422048
dtype: float64

In [23]:
perf_df.mean()[h3i_idx]

init_h3i    29.799482
qrad_h3i     5.579852
qknn_h3i     5.737786
dtype: float64

In [34]:
mean_tbl = pd.DataFrame(data=[
    perf_df.mean()[geo_idx].to_numpy(),
    perf_df.mean()[h3i_idx].to_numpy()
], columns=["Initialization", "Radius", "KNN"], index=["TI", "H3"])

In [35]:
mean_tbl

,Initialization,Radius,KNN
TI,28.962635,15.273070,7.422048
H3,29.799482,5.579852,5.737786


In [36]:
std_tbl = pd.DataFrame(data=[
    perf_df.std()[geo_idx].to_numpy(),
    perf_df.std()[h3i_idx].to_numpy()
], columns=["Initialization", "Radius", "KNN"], index=["TI", "H3"])

In [38]:
std_tbl

,Initialization,Radius,KNN
TI,0.359285,1.623289,0.742955
H3,0.721261,1.158461,1.204939
